In [18]:
import pandas as pd
import numpy as np

lab_clean = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\testing.csv", sep = "|")

C:\Users\mhuertas\AppData\Local\Temp\ipykernel_18076\3073547063.py:4: DtypeWarning: Columns (1,7,8,9,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_clean = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\testing.csv", sep = "|")


In [55]:
# -----------------------------------------
# ----- Step 1: Results with reportable limits -> flag implausible results

lab_clean[lab_clean['peticio_id'].str.contains("-", na=False)]

,codi_p,peticio_id,any,data,codi_prova,prova,resultat,unitat_mesura,ref_min,ref_max,clean_result,clean_unit,comentari,comentari_unitat,num_type
713100,5977,20171229-5669850,2017,2017-12-29,B08258,UROCULTIU CONVENCIONAL (orina),"NEGATIU S'aillen menys de 10,000 UFC/ml de div...",NaN,NaN,NaN,negatiu,NaN,literal,NaN,NaN
714320,2878,20160527-5677037,2016,2016-05-27,B08258,UROCULTIU CONVENCIONAL (orina),"POSITIU S'aillen més de 100,000 UFC/ML de:",NaN,NaN,NaN,positiu,NaN,literal,NaN,NaN
714384,6593,20180601-5634915,2018,2018-06-01,B08258,UROCULTIU CONVENCIONAL (orina),NEGATIU NEGATIU,NaN,NaN,NaN,negatiu,NaN,literal,NaN,NaN
715268,5977,20151120-5667058,2015,2015-11-20,B08258,UROCULTIU CONVENCIONAL (orina),NEGATIU NEGATIU,NaN,NaN,NaN,negatiu,NaN,literal,NaN,NaN
716578,8333,20190913-5662309,2019,2019-09-13,B08258,UROCULTIU CONVENCIONAL (orina),NEGATIU NEGATIU,NaN,NaN,NaN,negatiu,NaN,literal,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7809347,5967,20140131-5665945,2014,2014-01-31,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,"0,8067",%,0.0,2.0,0.8067,%,NaN,done,n1
7813934,8572,20160527-5678611,2016,2016-05-27,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,"0,5758",%,0.0,2.0,0.5758,%,NaN,done,n1
7814759,6035,20140919-5635001,2014,2014-09-19,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,"0,8139",%,0.0,2.0,0.8139,%,NaN,done,n1
7815167,2379,20141211-5626685,2014,2014-12-03,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,"0,6381",%,0.0,2.0,0.6381,%,NaN,done,n1


In [56]:
lab_clean[lab_clean['peticio_id'] == "20171229-5669850"]

,codi_p,peticio_id,any,data,codi_prova,prova,resultat,unitat_mesura,ref_min,ref_max,clean_result,clean_unit,comentari,comentari_unitat,num_type
713100,5977,20171229-5669850,2017,2017-12-29,B08258,UROCULTIU CONVENCIONAL (orina),"NEGATIU S'aillen menys de 10,000 UFC/ml de div...",NaN,NaN,NaN,negatiu,NaN,literal,NaN,NaN
1327000,5977,20171229-5669850,2017,2017-12-29,Q75558,ALBÚMINA (ORINA); FR.MASSA. «PART DE CONJUNT»,199,mg/l,NaN,NaN,199.0,mg/L,NaN,done,n1
1327841,5977,20171229-5669850,2017,2017-12-29,Q13285,COLESTEROL; C.SUBST. (sèrum),171,mg/dl,NaN,NaN,171.0,mg/dL,NaN,done,n1
1600952,5977,20171229-5669850,2017,2017-12-29,Q46585,PROTEÏNA (SÈRUM); C.MASSA,75,g/l,60,83,75.0,g/L,NaN,done,n1
1614978,5977,20171229-5669850,2017,2017-12-29,Q02485,ALBÚMINA (SÈRUM); C.MASSA,39,g/l,32,46,39.0,g/L,NaN,done,n1
1627748,5977,20171229-5669850,2017,2017-12-29,Q30385,FOSFATASA ALCALINA (SÈRUM); C.CAT.,84,U/l,40,150,84.0,U/l,NaN,NaN,n1
1627754,5977,20171229-5669850,2017,2017-12-29,Q31585,'-GLUTAMIL TRANSFERASA (SÈRUM); C.CAT.,15,U/l,9,36,15.0,U/l,NaN,NaN,n1
1629434,5977,20171229-5669850,2017,2017-12-29,Q35585,IÓ SODI (SÈRUM); C.SUBST. «PART DE CONJUNT»,141,mmol/l,135,145,141.0,mmol/L,NaN,done,n1
1633406,5977,20171229-5669850,2017,2017-12-29,Q09385,CALCI (SÈRUM); C.SUBST.,"9,7",mg/dl,8.4,10.2,9.7,mg/dL,NaN,done,n1
1633416,5977,20171229-5669850,2017,2017-12-29,Q12785,COLESTEROL D'HDL; C. SUBST. (sèrum),51,mg/dl,45,65,51.0,mg/dL,NaN,done,n1


In [22]:
lab_clean[lab_clean['num_type'] == "other"]

,codi_p,peticio_id,any,data,codi_prova,prova,resultat,unitat_mesura,ref_min,ref_max,clean_result,clean_unit,comentari,comentari_unitat,num_type
1625701,6117,62211009594,2015,2015-06-19,Q26859,EXCRECIÓ DE PROTEÏNA (ORINA 24H); CABAL MASSA,"1,046,50",mg/24 h,0,300,"1,046,50",mg/(24.h),NaN,done,other
1640228,7237,80411009902,2015,2015-07-30,T55259,EXCRECIÓ DE CREATININI; C.SUBST (orina 24 hores),"1,314,78",mg/24 h,950.0,2490,"1,314,78",mg/(24.h),NaN,done,other
